### 1) Data Preparation

In [95]:
import pandas as pd
import numpy as np
import zipfile
import os
import datetime

In [96]:
def combine_zipped_data(root_data_folder_path):
    '''
    This function opens all zip files in a given folder and combines any csv data found
    within them into a single Pandas DataFrame.
    
    Parameters
    ----------
    root_data_folder_path: A string containing the path to a folder containing zip files
                           with csv data.
    
    Returns
    -------
    A Pandas Dataframe with all csv data found in the given folder, combined together along
    the index (0) axis.
    '''
    # Creating empty list of DataFrames
    data_to_combine = []
    # Looping through raw data folder
    with os.scandir(root_data_folder_path) as root_data_folder:
        total_files = len(os.listdir(root_data_folder_path))
        current_progress = 0
        for entry in root_data_folder:
            # Displaying current progress
            current_progress += 1
            print(f"Processing file {current_progress}/{total_files} ...", end="\r")
            # Searching for zipped data
            if entry.name.endswith(".zip") and entry.is_file():
                # Opening zipped data folders
                with zipfile.ZipFile(root_data_folder_path + '/' + entry.name, "r") as zipped:
                    for name in zipped.namelist():
                        # Searching for csv files in zipped folders
                        if name.endswith('.csv'):
                            with zipped.open(name) as delay_data:
                                # Reading csv and adding to list of datasets
                                data_to_combine.append(pd.read_csv(delay_data, low_memory=False))
    # Attempting to combine and return collected data
    print('All files unpacked. Combining data...', end='\r')
    combined_data = pd.concat(data_to_combine)
    print('Data successfully combined!           ')
    return combined_data

flight_df = combine_zipped_data("BTS_Data")
flight_df = flight_df.dropna(subset=["FlightDate","DepTime"])

Data successfully combined!           


In [105]:
# Removing all columns that are more than 5% NaN values 
flight_df = flight_df[flight_df.columns[flight_df.isna().sum() < flight_df.shape[0] / 20]]
# Dropping rows of data with NaN in the target delay column (ArrDel15)
flight_df = flight_df.dropna(subset="ArrDel15")

In [170]:
def combine_weather_data(root_data_folder_path):
    '''
    
    '''
    data_to_combine = []
    with os.scandir(root_data_folder_path) as root_data_folder:
        total_files = len(os.listdir(root_data_folder_path))
        current_progress = 0
        for entry in root_data_folder:
            # Displaying current progress
            current_progress += 1
            print(f"Processing file {current_progress}/{total_files} ...", end="\r")
            # Searching for csv data
            if entry.is_file() and entry.name[-4:] == ".csv":
                # Collecting csv files for combination
                airport_df = pd.read_csv(entry.path)
                airport_df.loc[:,"airport_code"] = entry.name[:-4]
                airport_df = airport_df.fillna(value={"gust":0})
                data_to_combine.append(airport_df)
    # Attempting to combine and return collected data
    print('All files unpacked. Combining data...', end='\r')
    combined_data = pd.concat(data_to_combine)
    print('Data successfully combined!           ')
    return combined_data

root_data_folder_path = "WeatherData_Clean"
weather_df = combine_weather_data(root_data_folder_path)

Data successfully combined!           


In [178]:
# Dropping extremely small number of rows with null windspeed/dewpoint/temp values
weather_df = weather_df.dropna()

In [183]:
def attach_origin_weather_data(flight_df, weather_df):
    '''
    
    '''
    flight_df.loc[:,weather_df.columns] = np.NaN
    n_airports_to_inspect = len(flight_df.Origin.unique())
    airports_with_weather = weather_df.airport_code.unique()
    # Looping over all airport codes
    for ind, airport_code in enumerate(list(flight_df.Origin.unique())):
        # Checking if weather data exists for given airport
        if airport_code in airports_with_weather:
            print(f'Currently processing airport code {airport_code}. {ind}/{n_airports_to_inspect}  ', end='\r')
            # Looping over all flights by airport origin code
            airport_flight_df = flight_df[flight_df.Origin == airport_code]
            airport_weather_df = weather_df[weather_df.airport_code == airport_code]
            # Getting relative flight departure date times
            airport_date_dt = airport_flight_df.FlightDate.apply(lambda datestring: datetime.datetime.strptime(datestring, "%Y-%m-%d").timestamp())
            airport_date_dt = airport_date_dt + airport_flight_df.DepTime
            # Getting relative weather data times
            weather_date_dt = airport_weather_df.record_start_date.apply(lambda datestring: datetime.datetime.strptime(datestring, "%Y-%m-%d %H:%M:%S").timestamp())
            # Getting differences in times
            date_diff = airport_date_dt - weather_date_dt
            for entry_ind, entry_time in enumerate(weather_date_dt):
                #if entry_ind % 100 == 1:
                #    print(f"Currently processing entry number {entry_ind}")
                # Finding closest weather collection time to current flight entry
                date_diff = airport_date_dt - entry_time
                smallest_diff_ind = date_diff.tolist().index(min(date_diff))
                # Finding associated row of closest weather collection time
                closest_weather_row = airport_weather_df.iloc[smallest_diff_ind,:]
                # Finding associated row of current flight entry
                flight_ind = airport_flight_df.index[entry_ind]
                # Attaching weather row to flight row
                flight_df.loc[flight_ind,weather_df.columns] = closest_weather_row
    print("Data successfully attached!                                       ", end="\r")
    return flight_df

# Restricting attachment to only airports with known weather data
relevant_flights_df = flight_df[flight_df.Origin.isin(weather_df.airport_code.unique())]
# Subsetting dataset for demo testing purposes (NOT FINAL)
relevant_flights_df = relevant_flights_df[relevant_flights_df.Origin == "SEA"]
relevant_flights_df = relevant_flights_df[relevant_flights_df.Year == 2022]
relevant_flights_df = relevant_flights_df[relevant_flights_df.Month.isin((5,6,7))]
test = attach_origin_weather_data(relevant_flights_df,weather_df)

C:\Users\bruno\AppData\Local\Temp\ipykernel_39604\1582042063.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'KSEA' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  flight_df.loc[flight_ind,weather_df.columns] = closest_weather_row
C:\Users\bruno\AppData\Local\Temp\ipykernel_39604\1582042063.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Seattle' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  flight_df.loc[flight_ind,weather_df.columns] = closest_weather_row
C:\Users\bruno\AppData\Local\Temp\ipykernel_39604\1582042063.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'D' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  flight_df.loc[fli

In [92]:
flight_df[flight_df.Origin.isin(weather_df.airport_code.unique())].shape

(85790, 147)

In [75]:
flight_df.head(2000).origin.unique().shape

(38,)

In [72]:
flight_df.columns.tolist()

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'FlightDate',
 'Marketing_Airline_Network',
 'Operated_or_Branded_Code_Share_Partners',
 'DOT_ID_Marketing_Airline',
 'IATA_Code_Marketing_Airline',
 'Flight_Number_Marketing_Airline',
 'Originally_Scheduled_Code_Share_Airline',
 'DOT_ID_Originally_Scheduled_Code_Share_Airline',
 'IATA_Code_Originally_Scheduled_Code_Share_Airline',
 'Flight_Num_Originally_Scheduled_Code_Share_Airline',
 'Operating_Airline ',
 'DOT_ID_Operating_Airline',
 'IATA_Code_Operating_Airline',
 'Tail_Number',
 'Flight_Number_Operating_Airline',
 'OriginAirportID',
 'OriginAirportSeqID',
 'OriginCityMarketID',
 'Origin',
 'OriginCityName',
 'OriginState',
 'OriginStateFips',
 'OriginStateName',
 'OriginWac',
 'DestAirportID',
 'DestAirportSeqID',
 'DestCityMarketID',
 'Dest',
 'DestCityName',
 'DestState',
 'DestStateFips',
 'DestStateName',
 'DestWac',
 'CRSDepTime',
 'DepTime',
 'DepDelay',
 'DepDelayMinutes',
 'DepDel15',
 'DepartureDelayGroups',
 '

In [22]:
flight_df.columns.tolist()

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'FlightDate',
 'Marketing_Airline_Network',
 'Operated_or_Branded_Code_Share_Partners',
 'DOT_ID_Marketing_Airline',
 'IATA_Code_Marketing_Airline',
 'Flight_Number_Marketing_Airline',
 'Originally_Scheduled_Code_Share_Airline',
 'DOT_ID_Originally_Scheduled_Code_Share_Airline',
 'IATA_Code_Originally_Scheduled_Code_Share_Airline',
 'Flight_Num_Originally_Scheduled_Code_Share_Airline',
 'Operating_Airline ',
 'DOT_ID_Operating_Airline',
 'IATA_Code_Operating_Airline',
 'Tail_Number',
 'Flight_Number_Operating_Airline',
 'OriginAirportID',
 'OriginAirportSeqID',
 'OriginCityMarketID',
 'Origin',
 'OriginCityName',
 'OriginState',
 'OriginStateFips',
 'OriginStateName',
 'OriginWac',
 'DestAirportID',
 'DestAirportSeqID',
 'DestCityMarketID',
 'Dest',
 'DestCityName',
 'DestState',
 'DestStateFips',
 'DestStateName',
 'DestWac',
 'CRSDepTime',
 'DepTime',
 'DepDelay',
 'DepDelayMinutes',
 'DepDel15',
 'DepartureDelayGroups',
 '

In [23]:
flight_df.head()[["FlightDate","DepTime"]]

,FlightDate,DepTime
0,2022-05-03,1018.0
1,2022-05-04,1002.0
2,2022-05-01,2039.0
3,2022-05-02,2036.0
4,2022-05-03,2035.0


In [54]:
# Getting relative flight departure date times
flight_date_dt = flight_df.FlightDate.apply(lambda datestring: datetime.datetime.strptime(datestring, "%Y-%m-%d").timestamp())
flight_date_dt = flight_date_dt + flight_df.DepTime
# Getting relative weather data times
weather_date_dt = weather_df.record_start_date.apply(lambda datestring: datetime.datetime.strptime(datestring, "%Y-%m-%d %H:%M:%S").timestamp())



In [87]:
type(weather_date_dt - flight_date_dt[0])

pandas.core.series.Series

In [56]:
sum(np.isnan(weather_date_dt))

0

In [57]:
sum(np.isnan(flight_date_dt))

0

In [67]:
weather_df.airport_code

0        BIL
1        BIL
2        BIL
3        BIL
4        BIL
        ... 
20244    TWF
20245    TWF
20246    TWF
20247    TWF
20248    TWF
Name: airport_code, Length: 720123, dtype: object

In [109]:
test.columns.tolist()

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'FlightDate',
 'Marketing_Airline_Network',
 'Operated_or_Branded_Code_Share_Partners',
 'DOT_ID_Marketing_Airline',
 'IATA_Code_Marketing_Airline',
 'Flight_Number_Marketing_Airline',
 'Operating_Airline ',
 'DOT_ID_Operating_Airline',
 'IATA_Code_Operating_Airline',
 'Tail_Number',
 'Flight_Number_Operating_Airline',
 'OriginAirportID',
 'OriginAirportSeqID',
 'OriginCityMarketID',
 'Origin',
 'OriginCityName',
 'OriginState',
 'OriginStateFips',
 'OriginStateName',
 'OriginWac',
 'DestAirportID',
 'DestAirportSeqID',
 'DestCityMarketID',
 'Dest',
 'DestCityName',
 'DestState',
 'DestStateFips',
 'DestStateName',
 'DestWac',
 'CRSDepTime',
 'DepTime',
 'DepDelay',
 'DepDelayMinutes',
 'DepDel15',
 'DepartureDelayGroups',
 'DepTimeBlk',
 'TaxiOut',
 'WheelsOff',
 'WheelsOn',
 'TaxiIn',
 'CRSArrTime',
 'ArrTime',
 'ArrDelay',
 'ArrDelayMinutes',
 'ArrDel15',
 'ArrivalDelayGroups',
 'ArrTimeBlk',
 'Cancelled',
 'Diverted',
 'C

In [111]:
test.isna().sum().tolist()

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 26385,
 26385,
 26385,
 26385,
 26385,
 26385,
 26385,
 26385,
 26385,
 26385,
 45531,
 26385,
 26385,
 26385,
 26385,
 26385,
 26385,
 26385,
 26385,
 26385,
 26385,
 26385,
 26385,
 26385,
 26385,
 26385,
 26385]

In [147]:
test.loc[:,weather_df.columns].head(1).iloc[:,5:]

,day_ind,temp,dewPt,rh,wdir_cardinal,gust,wspd,pressure,precip_hrly,wx_phrase,...,start_day,start_month,start_year,start_isoweekday,start_hour_gmt,start_minute_gmt,record_end_date,end_hour_gmt,end_minute_gmt,airport_code
5693,D,55.0,48.0,77.0,NNE,NaN,7.0,29.55,0.0,Partly Cloudy,...,1.0,6.0,2022.0,2.0,11.0,53.0,2022-06-01 13:53:00,13.0,53.0,SEA


In [131]:
test.columns

Index(['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate',
       'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners',
       'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline',
       'Flight_Number_Marketing_Airline', 'Operating_Airline ',
       'DOT_ID_Operating_Airline', 'IATA_Code_Operating_Airline',
       'Tail_Number', 'Flight_Number_Operating_Airline', 'OriginAirportID',
       'OriginAirportSeqID', 'OriginCityMarketID', 'Origin', 'OriginCityName',
       'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac',
       'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'Dest',
       'DestCityName', 'DestState', 'DestStateFips', 'DestStateName',
       'DestWac', 'CRSDepTime', 'DepTime', 'DepDelay', 'DepDelayMinutes',
       'DepDel15', 'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOut',
       'WheelsOff', 'WheelsOn', 'TaxiIn', 'CRSArrTime', 'ArrTime', 'ArrDelay',
       'ArrDelayMinutes', 'ArrDel15', 'ArrivalDelayGroups'

In [151]:
demo_df = test.drop("gust", axis=1).dropna()

In [153]:
demo_df.isna().sum().sum()

0

In [155]:
demo_df.to_pickle("combined_flight_data")

In [161]:
test.describe().iloc[:,-15:]

,temp,dewPt,rh,gust,wspd,pressure,precip_hrly,start_day,start_month,start_year,start_isoweekday,start_hour_gmt,start_minute_gmt,end_hour_gmt,end_minute_gmt
count,19146.0,19146.0,19146.0,0.0,19146.0,19146.00,19146.0,19146.0,19146.0,19146.0,19146.0,19146.0,19146.0,19146.0,19146.0
mean,55.0,48.0,77.0,NaN,7.0,29.55,0.0,1.0,6.0,2022.0,2.0,11.0,53.0,13.0,53.0
std,0.0,0.0,0.0,NaN,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,55.0,48.0,77.0,NaN,7.0,29.55,0.0,1.0,6.0,2022.0,2.0,11.0,53.0,13.0,53.0
25%,55.0,48.0,77.0,NaN,7.0,29.55,0.0,1.0,6.0,2022.0,2.0,11.0,53.0,13.0,53.0
50%,55.0,48.0,77.0,NaN,7.0,29.55,0.0,1.0,6.0,2022.0,2.0,11.0,53.0,13.0,53.0
75%,55.0,48.0,77.0,NaN,7.0,29.55,0.0,1.0,6.0,2022.0,2.0,11.0,53.0,13.0,53.0
max,55.0,48.0,77.0,NaN,7.0,29.55,0.0,1.0,6.0,2022.0,2.0,11.0,53.0,13.0,53.0


In [179]:
weather_df[weather_df.airport_code == "SEA"].isna().sum()

Unnamed: 0           0
obs_id               0
obs_name             0
valid_time_gmt       0
expire_time_gmt      0
day_ind              0
temp                 0
dewPt                0
rh                   0
wdir_cardinal        0
gust                 0
wspd                 0
pressure             0
precip_hrly          0
wx_phrase            0
location_id          0
record_start_date    0
start_day            0
start_month          0
start_year           0
start_isoweekday     0
start_hour_gmt       0
start_minute_gmt     0
record_end_date      0
end_hour_gmt         0
end_minute_gmt       0
airport_code         0
dtype: int64

In [171]:
sum(weather_df[weather_df.airport_code == "SEA"].gust > 0)

1638

In [180]:
weather_df.shape

(707304, 27)

In [182]:
flight_df.isna().sum().sum()

0